In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from osc_toolkit import oscillation, delayed_oscillation
from models import *
import matplotlib.ticker as mticker

In [2]:
def delayed_full_first_order_model(delays):
    td1, td2 = delays
    alpha, beta, theta, phi, ep, k, lam, m = symbols('alpha beta theta phi ep k lam m')
    cA = 2 * (1 - y(0)) - lam * (y(1) + y(1)**m)
    dcA2dt = cA * y(2) - alpha * y(0) * y(1, t-td2)**m - theta * y(0)
    dcSdt = alpha/lam * y(0) * y(1, t-td2)**m - beta * \
        y(1, t-td1)**m + theta/lam * y(0) - phi * y(1)
    dcOdt = ep * (1 - y(2) * cA - k * y(2) * y(1, t-td2)**m) 
    dde = jitcdde([dcA2dt, dcSdt, dcOdt], control_pars=[
                  alpha, beta, theta, phi, ep, k, lam, m])
    return dde

def calc_all_delayed_full_model(sol, consts, *params):
    lam, m = consts
    cA2 = sol[:, 0]
    cS = sol[:, 1]
    cO = sol[:, 2]
    cM = cS ** m
    cA = 2 * (1 - cA2) - lam * (cS + cM)
    return np.array([cA2, cS + cM, cA, cO])

delayed_full_first_order_consumeO = {'model': delayed_full_first_order_model, 'calc_all': calc_all_delayed_full_model,
                            'info': '6 params: alpha, beta, theta, phi, epsilon, k. 2 consts: lam, m, 3 vars: cA2, cS, cO'}

In [3]:
#parameters
rs_ac, rds_ac, rs, rds, ro, k = 1450, 75, 11, 300, 1e3, 10
rext = 12.6
lam, m = 0.11, 2
td1, td2 = 0.01, 0.03

#Definition of the model
alpha, beta, theta, phi, ep = rs_ac / rext, rds_ac / rext, rs / rext, rds / rext, ro / rext
params = np.array([alpha, beta, theta, phi, ep, k])
consts = np.array([lam, m])
delay = np.array([td1, td2])
init_cond = np.array([0, 0, 0])
model = delayed_oscillation(delayed_full_first_order_consumeO, delay=delay, params=params, consts=consts, init_cond=init_cond)

In [5]:
model.interactive_plot(t=5, nvars=3, ran=200)

interactive(children=(FloatSlider(value=115.07936507936508, description='alpha', max=315.07936507936506, step=…